<a href="https://colab.research.google.com/github/louistrue/learn-ifc/blob/main/BFH-25-simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Einrichtung und Installation
Bevor wir mit der Bearbeitung von IFC-Modellen beginnen, müssen wir sicherstellen, dass `ifcopenshell` und `lark-parser` installiert sind. Diese Bibliotheken ermöglichen es uns, IFC-Dateien effektiv zu bearbeiten und zu analysieren.


In [ ]:
%pip install ifcopenshell
%pip install lark-parser

### Erstellen eines neuen IFC-Modells
Hier initialisieren wir ein neues IFC-Modell und erstellen ein Grundelement, das `IfcProject`, welches für die Organisation und Speicherung unserer Bauinformationen notwendig ist.

Nach dem Aufbau der Grundstruktur des Modells konstruieren wir eine Wand mit spezifischen Massen und ordnen diese der Etage zu. Ausserdem wird das Material für die Wand festgelegt.

Nachdem das IFC-Modell erfolgreich erstellt und gespeichert wurde, stellen wir es zur Verfügung, damit es heruntergeladen wird.

In [ ]:
import ifcopenshell
from ifcopenshell.api import run
import datetime


# Ein leeres IFC-Modell erstellen
model = ifcopenshell.file()

# Jedes Projekt muss ein IfcProject-Element enthalten. Hier wird das Projekt erstellt.
project = run("root.create_entity", model, ifc_class="IfcProject", name="Mein Projekt")

# Metrische Einheiten zuweisen
run("unit.assign_unit", model)

# Einen geometrischen Kontext erstellen, um 3D-Geometrien zu unterstützen
context = run("context.add_context", model, context_type="Model")
body = run("context.add_context", model, context_type="Model", context_identifier="Body", target_view="MODEL_VIEW", parent=context)

# Hierarchie erstellen: Standort, Gebäude und Etage
site = run("root.create_entity", model, ifc_class="IfcSite", name="Projektstandort")
building = run("root.create_entity", model, ifc_class="IfcBuilding", name="Gebaeude A")
storey = run("root.create_entity", model, ifc_class="IfcBuildingStorey", name="Erdgeschoss")

# Objekte der Hierarchie zuordnen: Standort zum Projekt, Gebäude zum Standort, Etage zum Gebäude
run("aggregate.assign_object", model, relating_object=project, products=[site])
run("aggregate.assign_object", model, relating_object=site, products=[building])
run("aggregate.assign_object", model, relating_object=building, products=[storey])

# Eine Wand erstellen mit spezifischen Dimensionen und dieser zur Etage zuordnen
wall = run("root.create_entity", model, ifc_class="IfcWall", name="Meine coole Wand")
wall_placement = run("geometry.edit_object_placement", model, product=wall)
representation = run("geometry.add_wall_representation", model, context=body, length=5, height=3, thickness=0.2)
wall_representation = run("geometry.assign_representation", model, product=wall, representation=representation)

# Material der Wand zuweisen
material = model.createIfcMaterial("Beton")
material_layer = model.createIfcMaterialLayer(material, 200, None)  # Dicke in Millimeter
material_layer_set = model.createIfcMaterialLayerSet([material_layer], None)
material_layer_set_usage = model.createIfcMaterialLayerSetUsage(material_layer_set, "AXIS2", "POSITIVE", -100)  # Offset von der Mitte
model.createIfcRelAssociatesMaterial(ifcopenshell.guid.new(), None, "Material zu Wand", None, [wall], material_layer_set_usage)

# Die Wand der räumlichen Struktur (Etage) zuordnen
rel_contained_in_spatial_structure = model.createIfcRelContainedInSpatialStructure(
    GlobalId=ifcopenshell.guid.new(),
    RelatedElements=[wall],
    RelatingStructure=storey
)

# Modell speichern
current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")  # Zeitstempel bis zur Sekunde
filename = f"einfache_Wand_{current_time}.ifc"
model.write(filename)
print(f"Das Modell wurde erfolgreich erstellt und unter '{filename}' gespeichert.")

# from google.colab import files
# files.download(filename)

### Fensterplatzierung
Nachdem wir die Wand erstellt haben, öffnen wir die zuvor erstellte Datei und fügen ein Fenster hinzu.

Nachdem das Fenster erfolgreich erstellt und gespeichert wurde, stellen wir es zur Verfügung, damit es heruntergeladen und in anderen Anwendungen verwendet werden kann.

In [ ]:
import ifcopenshell

# IFC-Modell laden
model = ifcopenshell.open(filename)
storey = model.by_type("IfcBuildingStorey")[0] if model.by_type("IfcBuildingStorey") else None

if not storey:
    print("Keine Gebäudeetage im Modell gefunden.")
    exit()

# Wand finden, auf der das Fenster platziert wird
walls = model.by_type("IfcWall")
if not walls:
    print("Keine Wand im Modell gefunden.")
    exit()

wall = walls[0]

# Parameter für das Fenster festlegen
window_params = {
    "name": "Neues Fenster",
    "breite": 1800,
    "tiefe": 250,
    "hoehe": 1200,
    "offset_x": 1500,
    "offset_y": 50,
    "offset_z": 500
}

# Platzierung des Fensters relativ zur Wand
wall_placement = wall.ObjectPlacement
wall_location = wall_placement.RelativePlacement.Location
placement_point = model.createIfcCartesianPoint((wall_location.Coordinates[0] + window_params["offset_x"]/1000.0,
                                                 wall_location.Coordinates[1] + window_params["offset_y"]/1000.0,
                                                 wall_location.Coordinates[2] + window_params["offset_z"]/1000.0))
axis2placement = model.createIfcAxis2Placement3D(placement_point)
window_placement = model.createIfcLocalPlacement(wall_placement, axis2placement)

# Fenstergeometrie erstellen
# Create proper 2D axis placement for the profile
profile_point = model.createIfcCartesianPoint((0.0, 0.0))
profile_axis = model.createIfcAxis2Placement2D(profile_point)
rect_profile = model.createIfcRectangleProfileDef("AREA", "BaseProfile", profile_axis, window_params["breite"]/1000.0, window_params["tiefe"]/1000.0)
extrude_dir = model.createIfcDirection((0.0, 0.0, 1.0))
extruded_area = model.createIfcExtrudedAreaSolid(rect_profile, profile_axis, extrude_dir, window_params["hoehe"]/1000.0)
rep = model.createIfcShapeRepresentation(model.by_type("IfcGeometricRepresentationContext")[0], "Body", "SweptSolid", [extruded_area])
product_def_shape = model.createIfcProductDefinitionShape(None, None, [rep])

# Fensterentität erstellen
new_window = model.createIfcWindow(ifcopenshell.guid.new(), None, "Neues Fenster", None, None, window_placement, product_def_shape, None)
rel_contained = model.createIfcRelContainedInSpatialStructure(ifcopenshell.guid.new(), None, "Contains", None, [new_window], storey)

# Normalisieren der RGB-Werte von 0-255 auf 0-1
color = [100 / 255.0, 149 / 255.0, 237 / 255.0]  # Kornblumenblau

# Erstellen eines neuen Stils in der IFC-Datei
style = ifcopenshell.api.run("style.add_style", model, name="DynamicStyle")

# Hinzufügen von Oberflächenstileigenschaften einschließlich der Farbe
ifcopenshell.api.run("style.add_surface_style", model, style=style, ifc_class="IfcSurfaceStyleShading", attributes={
    "SurfaceColour": {"Name": None, "Red": color[0], "Green": color[1], "Blue": color[2]}
})

# Zuweisen des Stils zur Formdarstellung des Fensters
ifcopenshell.api.run("style.assign_representation_styles", model, shape_representation=rep, styles=[style])

# Aktualisiertes Modell speichern
current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")  # New timestamp for updated model
filename = f"Wand_mit_Fenster_{current_time}.ifc"
model.write(filename)
print("Updated model saved to:", filename)

# from google.colab import files
# files.download(filename)

### Bereinigung des Arbeitsbereichs
Nachdem das Modell gespeichert und heruntergeladen wurde, ist es oft sinnvoll, den Arbeitsbereich aufzuräumen, indem man nicht mehr benötigte Dateien entfernt. Dies hilft, die Übersichtlichkeit zu bewahren und Speicherplatz freizugeben.

In [235]:
import os

# Get the list of all files and directories in the current working directory
file_list = os.listdir()

# Loop through the file list
for file_name in file_list:
    # Check if the file ends with '.ifc'
    if file_name.endswith('.ifc'):
        # Construct full file path
        file_path = os.path.join(os.getcwd(), file_name)
        # Remove the file
        os.remove(file_path)
        print(f"Removed {file_path}")
